In [ ]:
# IMPORT LIBRARIES

In [36]:
import os
import re
import nltk
import pandas as pd
import string
import Docket_Sheet_Classification_module as dsc_m

In [ ]:
# CREATE OBJECT FOR DOCKET SHEET EXCEL FILE

In [40]:
Docket_sheet_file = r'C:\Users\Chris.Cirelli\Desktop\Python Programming Docs\GSU\Sprint Project\Docket-Sheet-Classification\2018-02-18 Docket Review.xlsx'

In [ ]:
# CREATE A DATAFRAME WHOSE INDEX IS THE SET OF WORDS AND COLUMNS THE FREQ DIST FOR EACH STAGE OF THE LIFE CYCLE

In [39]:
def master_code_Pipeline_create_dataframe_setWord_freqDist(Docket_sheet_file):
    '''The purpose of this master code pipeline is to pipeline all of the functions required to create the word frequency distribution dataframe 
       for each life cycle of the Docket Sheet
       
       Input       = Docket sheet file as an excel spreadsheet.  We will read in using pandas. 
       Operations  = 1.) Import the docket sheet, delimit columns, remove rows where time-period = 0/None
                     2.) Concatenate all of the text located in the text column of the Docket Sheet for which a time period has been assigned.
                         Write the text file to current-working-directory.
                     3.) Import into memor the concatenated text file. 
                     4.) Tokenize text file, clean tokens of punctuation, stop words and limit to isalpha(). 
                     5.) Create a set of from our tokenized and cleaned text.  This set will constitute the basis for our word frequency distribution. 
                     6.) Create a dataframe whose index is the above create set of words.  Generate the word frequency for each life cycle stage for
                         this set of words. Return to the user the dataframe in memory. 
    '''
    
    # 1.) Import Target File / Delimit Columns / Remove Rows if Period = None
    df_Master_DocketSheet_File = dsc_m.import_docket_sheet_file(Docket_sheet_file)
    
    # 2.) Concatenate Text From Docket Sheet Time Periods
    dsc_m.create_Concatenated_text_file(df_Master_DocketSheet_File, 'Concatenate_DocketSheet_Text')
    
    # 3.) Import Into Memory Concatenated Text File
    Concat_txt_File = dsc_m.import_concatTxt('Concatenated_Docket_Text.txt')
    
    # 4.) Clean & Tokenize Concatenated Text File
    Clean_tokenized_concatText = dsc_m.clean_andTokenize_text(Concat_txt_File)
    
    # 5.) Create Set of Words From Clean & Tokenized Concat Text File
    Set_tokenized_concatText = list(set(Clean_tokenized_concatText))
    
    # 6.) Create & Return Word Frequency Distribution Dataframe
    return dsc_m.create_dataframe_setWord_freqDist(df_Master_DocketSheet_File, Set_tokenized_concatText)


In [ ]:
df_Word_Freq_Dist = master_code_Pipeline_create_dataframe_setWord_freqDist()

In [299]:
def write_to_excel(dataframe, filename):
    import pandas as pd
    writer = pd.ExcelWriter(filename+'.xlsx')
    dataframe.to_excel(writer, sheet_name = 'Data')
    writer.save()

In [303]:
write_to_excel(df_docketsheet_wordSet, 'DocketSheet_wordSet_Freq_Dist')


In [ ]:
    # Create a dictionary whose keys are the file classes and values the files for each group. 
    Dict_files_by_class = create_file_list_by_group(Excel_file)
    
    # Create a Master Dataframe to house final values
    Master_dataframe = pd.DataFrame({}, index = list(dataframe_unique_tokens.index))
    
    # Loop over the dictionary keys
    for Class in Dict_files_by_class.keys():
        
        # Create a list of the files
        File_list = list(Dict_files_by_class[Class])
        
        # Feed each class list trough the word counter
        df_single_class_word_count = get_frequencyDist_legal_text(File_list, dataframe_unique_tokens)
        
        # Merge individual text file columns into one percentage for the given class
        df_merged_columns = Merge_dataframe_columns_calc_percentage(Class, df_single_class_word_count)
        
        # Merge individual dataframes into master dataframe
        Master_dataframe[Class] = df_merged_columns[Class]
        
    # Return the Master Dataframe with the word % for all classes. 
    return Master_dataframe